<a href="https://colab.research.google.com/github/vitakot/strategy_carry/blob/main/CryptoCarry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -U datar[pandas]

In [ ]:
import pandas as pd
from datar.apis.base import head, tail, log
from datar.dplyr import inner_join, group_by, arrange, mutate, lead, lag
from datar.base import c
from datar import f

In [ ]:
def load_and_prepare_funding_data():
    futures = pd.read_feather('/content/drive/MyDrive/---Shared---/Data/RWResearch/ftx_futures_ohlc_btc_xrp_1h.feather')
    perp_rate = pd.read_feather('/content/drive/MyDrive/---Shared---/Data/RWResearch/ftx_perpetual_funding_rates_btc_xrp.feather')

    futures['date'] = pd.to_datetime(futures['date'], format='%Y-%m-%dT%H:%M:%S', utc=True)
    perp_rate['date'] = pd.to_datetime(perp_rate['date'], format='%Y-%m-%d %H:%M:%S', utc=True)

    joined = inner_join(futures, perp_rate, by=c('ticker', 'date'))
    grouped = group_by(joined, f.ticker)
    arranged = arrange(grouped, f.date)
    mutated = mutate(arranged, price_change=log(f.close / lag(f.close)), funding_accrual=-lead(f.rate))
    return mutated

In [ ]:
    funding_data = load_and_prepare_funding_data()
    print(head(funding_data).to_string())


     ticker                      date          open          high           low         close        volume      rate  price_change  funding_accrual
   <object>     <datetime64[ns, UTC]>     <float64>     <float64>     <float64>     <float64>     <float64> <float64>     <float64>        <float64>
0  BTC-PERP 2019-07-20 12:00:00+00:00  10564.250000  10630.000000  10525.500000  10539.500000  1.052997e+07  0.000011           NaN         0.000003
1  XRP-PERP 2019-07-20 12:00:00+00:00      0.327975      0.328400      0.327000      0.327975  0.000000e+00  0.000029      0.000877        -0.000028
2  XRP-PERP 2019-07-20 13:00:00+00:00      0.327975      0.331675      0.327025      0.328575  0.000000e+00  0.000028      0.006849        -0.000021
3  BTC-PERP 2019-07-20 13:00:00+00:00  10539.500000  10796.250000  10477.000000  10548.750000  2.361670e+07 -0.000003      0.002468        -0.000003
4  XRP-PERP 2019-07-20 14:00:00+00:00      0.328575      0.330100      0.327900      0.329375  0.000000e+0